# Applied Machine Learning: Assignment \#7

### Sheetal Parikh
EN.605.631.81<br>
March 15, 2021
***
***

*Continuing to the previous machine learning problem, let's get back to the pre-processed dataset Suicide Rates Overview 1985 to 2016 file. We would like to have a machine learning model to predict the suicide rate 'suicides/100k pop'.* 

## Problem 1
*Keep the variables as one-hot encoded and develop a multiple linear regression model. Use your model to predict the target variable for the people with age 20, male, and generation X. What is the MAE error of this prediction? How many line coefficients are there?*

### Preprocessing Data Set similar to Module 3 and 6 Homework

In [1]:
# imports for notebook
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns; sns.set(style="ticks", color_codes=True)
#from sklearn.model_selection import train_test_split
#from sklearn import datasets, metrics
from sklearn.linear_model import LinearRegression
import os 

#checking current directory
#print(os.getcwd() + "\n")

# change the current directory 
# to specified directory 
os.chdir(r"C:\Users\Sheetal\Documents\Sheetal") 

#Read in file
relative_path = 'datasets/master.csv'
df_org = pd.read_csv(relative_path)

# Sanity check
print(f'N rows={len(df_org)}, M columns={len(df_org.columns)}')

#print first few rows to visualize dataset
df_org.head()

N rows=27820, M columns=12


,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [2]:
#renaming suicides/100k pop column
df_org.rename(columns = {'suicides/100k pop':'suicides_per_100k'}, inplace = True) 

df_org.isnull().sum()

country                   0
year                      0
sex                       0
age                       0
suicides_no               0
population                0
suicides_per_100k         0
country-year              0
HDI for year          19456
 gdp_for_year ($)         0
gdp_per_capita ($)        0
generation                0
dtype: int64

In [3]:
print(f'Column Names:')

#printing column names
print(df_org.columns)

Column Names:
Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides_per_100k', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation'],
      dtype='object')


In [4]:
#dropping 2016 data because very few data points - data possibly incomplete for 2016
df_org2 = df_org[df_org.year != 2016]
print(df_org2.shape)

(27660, 12)


We started out with 27,820 data points and were able to elminate 160 data points due to incomplete data.

In [5]:
df_org2.corr(method = 'pearson')

,year,suicides_no,population,suicides_per_100k,HDI for year,gdp_per_capita ($)
year,1.000000,-0.002953,0.011583,-0.039800,0.366786,0.338854
suicides_no,-0.002953,1.000000,0.616105,0.307103,0.151399,0.062168
population,0.011583,0.616105,1.000000,0.008461,0.102943,0.083006
suicides_per_100k,-0.039800,0.307103,0.008461,1.000000,0.074279,0.002015
HDI for year,0.366786,0.151399,0.102943,0.074279,1.000000,0.771228
gdp_per_capita ($),0.338854,0.062168,0.083006,0.002015,0.771228,1.000000


No variable has a very strong correlation with suicides/100k pop, however, HDI for year appears to have the strongest of the other variables. Also, the number of suicides has a strong correlation with the population. Also, GDP per capita has a strong correlation with HDI.

In [6]:
#dropping features
df = df_org2.copy()
df.drop(columns='suicides_no', inplace=True)
df.drop(columns='population', inplace=True)
df.drop(columns='country', inplace=True)
df.drop(columns=' gdp_for_year ($) ', inplace=True)
df.drop(columns='country-year', inplace=True)
df.drop(columns='HDI for year', inplace=True)

#printing updated data set
df.head()

,year,sex,age,suicides_per_100k,gdp_per_capita ($),generation
0,1987,male,15-24 years,6.71,796,Generation X
1,1987,male,35-54 years,5.19,796,Silent
2,1987,female,15-24 years,4.83,796,Generation X
3,1987,male,75+ years,4.59,796,G.I. Generation
4,1987,male,25-34 years,3.28,796,Boomers


The HDI for year variable has too many null values so I believe using the mean per country may not be sufficient and should just be removed. Also, I decided to remove the country variable because I wanted to limit the number of columns that would be added to the dataset.  I do believe variables such as HDI or GDP per capita would have some impact on suicide rates so since I removed HDI, I decided to keep GDP per capita in it's place.

In [7]:
#checking if we eliminated the NaNs
df.isnull().sum()

year                  0
sex                   0
age                   0
suicides_per_100k     0
gdp_per_capita ($)    0
generation            0
dtype: int64

### One-Hot Encoding

In [8]:
#one hot coding formula from module 3 notebook
#pandas get_dummies function is one-hot-encoder

def encode_onehot(_df, f):
    _df2 = pd.get_dummies(_df[f], prefix='', prefix_sep='').max(level=0, axis=1).add_prefix(f+'-')
    df4 = pd.concat([_df, _df2], axis=1)
    df4 = df4.drop([f], axis=1)
    return df4

In [9]:
#copying dataframe
df_oneHot = df.copy()

#converting sex column
df_oneHot = encode_onehot(df_oneHot, 'sex')

#converting age column
df_oneHot = encode_onehot(df_oneHot, 'age')

#converting generation column
df_oneHot = encode_onehot(df_oneHot, 'generation')

In [10]:
#renaming columns names so easier to use
df_oneHot.rename(columns = {'sex-male':'sex_male', 
                           'age-15-24 years':'age_15_24',
                           'generation-Generation X':'generation_Generation_X'}, inplace = True) 
#print(df_oneHot.columns)

In [11]:
df_oneHot.head()

,year,suicides_per_100k,gdp_per_capita ($),sex-female,sex_male,age_15_24,age-25-34 years,age-35-54 years,age-5-14 years,age-55-74 years,age-75+ years,generation-Boomers,generation-G.I. Generation,generation_Generation_X,generation-Generation Z,generation-Millenials,generation-Silent
0,1987,6.71,796,0,1,1,0,0,0,0,0,0,0,1,0,0,0
1,1987,5.19,796,0,1,0,0,1,0,0,0,0,0,0,0,0,1
2,1987,4.83,796,1,0,1,0,0,0,0,0,0,0,1,0,0,0
3,1987,4.59,796,0,1,0,0,0,0,0,1,0,1,0,0,0,0
4,1987,3.28,796,0,1,0,1,0,0,0,0,1,0,0,0,0,0


The generation, sex, and age columns were onehot encoded.  We can go ahead building models and creating testing and training sets.

### Building Multiple Linear Regression Model

In [12]:
#testing and training set
X = df_oneHot.loc[:, df_oneHot.columns != 'suicides_per_100k'].values
y = df_oneHot.loc[:, df_oneHot.columns == 'suicides_per_100k'].values.ravel()

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

In [13]:
X.shape, y.shape

((27660, 16), (27660,))

In [14]:
#building regression model for onehot encoded dataset

linear_oneHot = LinearRegression()
linear_oneHot.fit(X,y)
y_pred = linear_oneHot.predict(X)

The entire dataset was used for the training and testing set to ensure the best fit for the model

### Predictions and  Evaluating Model

In [15]:
# MAE formula from module 7 notebook
def mae(_y, _y_pred):
    return (len(_y)**-1) * np.sum(np.abs(_y_pred-_y))

In [16]:
#comparing test sets with predicted values

comparison_oneHot = pd.DataFrame({'Real Values':y,'Predicted Values':y_pred})
comparison_oneHot

,Real Values,Predicted Values
0,6.71,17.626612
1,5.19,22.489399
2,4.83,2.793793
3,4.59,33.570011
4,3.28,20.609137
...,...,...
27655,2.96,6.345364
27656,2.58,14.133899
27657,2.17,7.266502
27658,1.67,-7.566316


Many of the predicted values appear to be very different to the real values

In [17]:
#MAE of one-hot encoded dataset

#R2 score
r2 = linear_oneHot.score(X,y)

#getting number of coefficients
coefficients = linear_oneHot.coef_

In [18]:
print(f"MAE of Linear Model using One_Hot Encoding: {mae(y, y_pred):.4f}\n")
print(f"R2_score = {r2:.4f}\n")
print(f"Intercept = {linear_oneHot.intercept_:.4f}\n")
print(f'There are {len(coefficients)} coefficients :' )
print(coefficients)

MAE of Linear Model using One_Hot Encoding: 10.1628

R2_score = 0.2964

Intercept = 258.6500

There are 16 coefficients :
[-1.22850841e-01  1.71472285e-05 -7.41640926e+00  7.41640926e+00
 -4.50423132e+00 -9.78746677e-01  2.33836871e+00 -1.34367764e+01
  4.47466951e+00  1.21067161e+01 -3.87526869e-01 -5.12116385e-01
  1.55432211e-01  2.01889689e+00  5.49694350e-01 -1.82438019e+00]


The model has only an R2 score of about 0.30 so only about 30% of the data fits the regression model.  The model using the one-hot encoded method has 16 coefficients.  Based on the MAE of 10.1628, the model's predictions differ from the actual data by about an absolute value of about 10.

### Grouping the Dataset for people who are age 20, male, and belong to generation X

In [19]:
grouped = df_oneHot.copy()

#grouping by age (age 20), generation (generation X), and sex (male)
grouped = grouped[(grouped.age_15_24 == 1) & 
                  (grouped.generation_Generation_X == 1) & 
                  (grouped.sex_male == 1)]

In [20]:
grouped.head()

,year,suicides_per_100k,gdp_per_capita ($),sex-female,sex_male,age_15_24,age-25-34 years,age-35-54 years,age-5-14 years,age-55-74 years,age-75+ years,generation-Boomers,generation-G.I. Generation,generation_Generation_X,generation-Generation Z,generation-Millenials,generation-Silent
0,1987,6.71,796,0,1,1,0,0,0,0,0,0,0,1,0,0,0
13,1988,5.33,769,0,1,1,0,0,0,0,0,0,0,1,0,0,0
28,1989,3.71,833,0,1,1,0,0,0,0,0,0,0,1,0,0,0
37,1992,3.41,251,0,1,1,0,0,0,0,0,0,0,1,0,0,0
48,1993,7.40,437,0,1,1,0,0,0,0,0,0,0,1,0,0,0


In [21]:
print(f"There are {len(grouped.index)} people who are age 20, male, and of generation X\n")

There are 1057 people who are age 20, male, and of generation X



In [22]:
#findng the mean year and GDP of the grouped dataset
year  = grouped['year'].values
GDP = grouped['gdp_per_capita ($)'].values

year_mean = round(np.mean(year),0)
GDP_mean = round(np.mean(GDP),0)

print(f"Mean year of people who are age 20, male, and of generation X = {year_mean:.4f}\n")
print(f"Mean GDP of people who are age 20, male, and of generation X = {GDP_mean:.4f}") 

Mean year of people who are age 20, male, and of generation X = 1993.0000

Mean GDP of people who are age 20, male, and of generation X = 10910.0000


The mean year and GDP of males who are 20 and of generation X will be used in the prediction input since we are not given a specific year or GDP

In [23]:
#prediction input - 2 dimensional array
input = np.array([[year_mean, GDP_mean, 0,1,1,0,0,0,0,0,0,0,1,0,0,0]])

#new prediction of the suicide rate of someone who is age 20, male and generation X
y_pred2 = linear_oneHot.predict(input)

print(f"Someone who is age 20, male, and of generation X would have a predicted suicide rate of: {y_pred2.item(0):.4f}")

Someone who is age 20, male, and of generation X would have a predicted suicide rate of: 17.0629


In [24]:
#taking the entire grouped dataset and 
y_grouped = grouped.loc[:, grouped.columns == 'suicides_per_100k'].values.ravel()
#grouped dataset without the suicides rate column
y_pred_grouped = grouped.loc[:, grouped.columns != 'suicides_per_100k'].values

#getting predictions of only those who are age 20, male, and of generation X
y_pred3 = linear_oneHot.predict(y_pred_grouped)

In [25]:
#finding MAE of the group of people who are age 20, male, and of generation x and the new hyperplane
print(f"MAE of Linear Model using One_Hot Encoding of Age 20, Male, Generation X Group: {mae(y_grouped, y_pred3):.4f}\n")

MAE of Linear Model using One_Hot Encoding of Age 20, Male, Generation X Group: 9.5401



Overall, the MAE of the entire dataset and predictions was 10.1628 when using one hot encoding. We also got a low R2 of about 30%. The MAE of the age 20, male, and generation group and predictions was better at 9.5401.  Someone who in that group is predicted to have a suicide rate of 17.0629 suicides/per 100k. We can now look whether the MAE and prediction changes when using a dataset using numerical encoding. The model had 16 coefficients.

## Problem 2

*Now use the original sex, age and generation variables in numerical form and develop a new model. Use your model to predict the target value for the people with age 20, male, and generation X. What is the MAE error of this prediction? How many line coefficients are there? (Note that for this step you have to think of a way of encoding the original nominal age feature and generation feature into numerical features.)*

### Numerical Form

In [26]:
# converting sex, age, and generation to numerical data - using method similar from Module 3 assignment solution

df_numerical = df.copy()

#G.I. Generation - 1
#Silent - 2
#Boomers - 3
#Generation X - 4
#Millenials - 5
#Generation Z - 6
df_numerical.generation.replace(['G.I. Generation','Silent','Boomers','Generation X','Millenials','Generation Z'], 
                        [1,2,3,4,5,6], inplace=True)

# 0 = female
# 1 = male
df_numerical.sex.replace(['female', 'male'], [0, 1], inplace=True)

#5-14 = 10
#15-24 = 20
#25-34 = 30
#35-54 = 45
#55-74 = 65
#75+ = 80
df_numerical.age.replace(['5-14 years','15-24 years','25-34 years','35-54 years','55-74 years','75+ years'], 
                 [10,20,30,45,65,80], inplace=True)

df_numerical.head(10)

,year,sex,age,suicides_per_100k,gdp_per_capita ($),generation
0,1987,1,20,6.71,796,4
1,1987,1,45,5.19,796,2
2,1987,0,20,4.83,796,4
3,1987,1,80,4.59,796,1
4,1987,1,30,3.28,796,3
5,1987,0,80,2.81,796,1
6,1987,0,45,2.15,796,2
7,1987,0,30,1.56,796,3
8,1987,1,65,0.73,796,1
9,1987,0,10,0.00,796,4


### Building Multiple Linear Regression Model

In [27]:
#testing and training set
X2 = df_numerical.loc[:, df_numerical.columns != 'suicides_per_100k'].values
y2 = df_numerical.loc[:, df_numerical.columns == 'suicides_per_100k'].values.ravel()

In [28]:
#checking shape of testing and training set
X2.shape, y2.shape

((27660, 5), (27660,))

In [29]:
#building model with numerical dataset

linear_numerical = LinearRegression()
linear_numerical.fit(X2,y2)
y_pred4 = linear_numerical.predict(X2)

### Predictions and Evaluating Model

In [30]:
#comparing test sets with predicted values of entire numerical dataset

comparison_numerical = pd.DataFrame({'Real Values':y2,'Predicted Values':y_pred4})
comparison_numerical

,Real Values,Predicted Values
0,6.71,15.486308
1,5.19,22.337028
2,4.83,0.653489
3,4.59,31.827122
4,3.28,18.237809
...,...,...
27655,2.96,4.724534
27656,2.58,14.270692
27657,2.17,10.011196
27658,1.67,-4.821623


The predicted values appear to be very different to the actual values but seem to be a little closer compared to the one-hot encoded dataset.

In [31]:
#getting r2 score
r2_numerical = linear_numerical.score(X2,y2)

#getting number of coefficients
coefficients_numerical = linear_numerical.coef_

In [32]:
print(f"MAE of Linear Model using Numerical Encoding: {mae(y2, y_pred4):.4f}\n")
print(f"R2_score = {r2_numerical:.4f}\n")
print(f"Intercept = {linear_numerical.intercept_:.4f}\n")
print(f'There are {len(coefficients_numerical)} coefficients :' )
print(coefficients_numerical)

MAE of Linear Model using Numerical Encoding: 10.3285

R2_score = 0.2796

Intercept = 193.7455

There are 5 coefficients :
[-9.97849110e-02  1.48328185e+01  2.69543696e-01  1.76108362e-05
 -5.60639421e-02]


The model has only an R2 score of about 0.28 so only about 28% of the data fits the regression model. The model using the one-hot encoded method has 5 coefficients. Based on the MAE of 10.3285, the model's predictions differ from the actual data by about an absolute value of about 10.

### Grouping the Dataset for people who are age 20, male, and belong to generation X

In [33]:
grouped2 = df_numerical.copy()

#grouping dataset to be people who are 20, male, and generation X
grouped2 = grouped2[(grouped2.sex == 1) & (grouped2.age == 20) & (grouped2.generation == 4)]

In [34]:
grouped2.head()

,year,sex,age,suicides_per_100k,gdp_per_capita ($),generation
0,1987,1,20,6.71,796,4
13,1988,1,20,5.33,769,4
28,1989,1,20,3.71,833,4
37,1992,1,20,3.41,251,4
48,1993,1,20,7.40,437,4


In [35]:
print(f"There are {len(grouped2.index)} people who are age 20, male, and of generation X\n")

There are 1057 people who are age 20, male, and of generation X



As we should, we get the same number of people in the age 20, male, and generation X group.

In [36]:
#prediction input - 2 dimensional array
input2 = np.array([[year_mean, 1, 20, GDP_mean, 4]])

#new prediction of the suicide rate of someone who is age 20, male and generation X
y_pred4 = linear_numerical.predict(input2)

print(f"Someone who is age 20, male, and of generation X would have a preidcted suicide rate of: {y_pred4.item(0):.4f}")

Someone who is age 20, male, and of generation X would have a preidcted suicide rate of: 15.0657


The suicide rate of 15.0657 is lower that what we predicted in problem 1

In [37]:
#taking the entire grouped dataset and 
y_grouped2 = grouped2.loc[:, grouped2.columns == 'suicides_per_100k'].values.ravel()

#grouped dataset without the suicides rate column
y_pred_grouped2 = grouped2.loc[:, grouped2.columns != 'suicides_per_100k'].values

#getting predictions of only those who are age 20, male, and of generation X
y_pred5 = linear_numerical.predict(y_pred_grouped2)


In [38]:
print(f"MAE of Linear Model using Numerical Encoding of Age 20, Male, Generation X Group: {mae(y_grouped2, y_pred5):.4f}\n")

MAE of Linear Model using Numerical Encoding of Age 20, Male, Generation X Group: 8.8951



Overall, the MAE of the entire dataset and predictions was 10.3285 when using numerical encoding. We also got a low R2 of about 28%. The MAE of the age 20, male, and generation group and predictions was better at 8.8951. Someone who in that group is predicted to have suicide rate of 15.0657 suicides/per 100k. The model had 5 coefficients.

## Problem 3

*Did you note any change in these two model performances?*

The MAE using the one-hot encoded dataset resulted in a slightly better MAE of 10.1628 compared to the numerical encoded MAE of 10.3285 for the overall datset. However for the age 20, male, generation X group, the numerically encoded model resulted in a better MAE of 8.8951 compared to 9.5401 for the one-hot encoded model. The R2 of the one-hot encoded datasaet was also slightly better at 30% compared to the R2 of 28% for the numerical dataset. The numerical model predicted a suicide rate of about 15 suicides/100K for the age 20, male, and generation group which was lower than the predicted rate of 17 suicides/100k. Overall, the performance and predictions of both the models was very close. Therefore, I would chose to use the numerical encoding method for any nominal variables that have ordinality such as age and generation as well as any binary variables.  Numerical encoding produces less coefficients making the model more interpretable and more efficient especially with very large datasets.  The numerical encoded model had only 5 coefficients compared to 16 for the one-hot encoded dataset.

## Problem 4

*What is the prediction for age 33, male and generation Alpha (i.e. the generation after generation Z)?*

### One-Hot Encoding

In [39]:
#need to create new dataframe for new subfeature - alpha
df_oneHot2 = df_oneHot.copy()

#adding a new column for generation alpha
#filled with a default of 0 since we do not have any data on people who 
df_oneHot2.insert(14, 'generation - Alpha', 0)

df_oneHot2.head()

,year,suicides_per_100k,gdp_per_capita ($),sex-female,sex_male,age_15_24,age-25-34 years,age-35-54 years,age-5-14 years,age-55-74 years,age-75+ years,generation-Boomers,generation-G.I. Generation,generation_Generation_X,generation - Alpha,generation-Generation Z,generation-Millenials,generation-Silent
0,1987,6.71,796,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
1,1987,5.19,796,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
2,1987,4.83,796,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
3,1987,4.59,796,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0
4,1987,3.28,796,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0


In [40]:
#testing and training set
X3 = df_oneHot2.loc[:, df_oneHot2.columns != 'suicides_per_100k'].values
y3 = df_oneHot2.loc[:, df_oneHot2.columns == 'suicides_per_100k'].values.ravel()

In [41]:
#building model
linear_oneHot2 = LinearRegression()
linear_oneHot2.fit(X3,y3)

LinearRegression()

In [42]:
grouped3 = df_oneHot2.copy()

#renaming 25-34 age column so easier to use
grouped3.rename(columns = {'age-25-34 years':'age_25_34'}, inplace = True) 

#making group of those who are male and age 33 (between 25 and 34)
grouped3 = grouped3[(grouped3.sex_male == 1) & (grouped3.age_25_34 == 1)]

#grouped3.head()

#finding mean year and GDP of males who are 33 (or between 25 and 34)
year_mean2 = round(np.mean(grouped3['year'].values),0)
GDP_mean2 =  round(np.mean(grouped3['gdp_per_capita ($)']),0)

print(f'Mean of year for males who are 33 = {year_mean2:.4f}')
print(f'Mean GDP for males who are 33 = {GDP_mean2:.4f}') 

Mean of year for males who are 33 = 2001.0000
Mean GDP for males who are 33 = 16816.0000


In [43]:
print(f"There are {len(grouped3.index)} people who are 33 year old males\n")

There are 2305 people who are 33 year old males



The mean year and GDP of males who are 33 (or between 25 and 34) will be used in the prediction input since we are not given a specific year or GDP.  There was no grouping of generation since we do not have any data on anyone who is of generation alpha.

In [44]:
#prediction input - 2-Dimensional Array
input3 = np.array([[year_mean2, GDP_mean2, 0,1,0,1,0,0,0,0,0,0,0,1,0,0,0]])

#new prediction of suicide rate of someone who is a 33 year old male
y_pred6 = linear_oneHot2.predict(input3)

print(f"A 33 year old male would have a predicted suicide rate of: {y_pred6.item(0):.4f}")

A 33 year old male would have a predicted suicide rate of: 19.5515


In [45]:
#taking the entire grouped datasaet
y_grouped3 = grouped3.loc[:, grouped3.columns == 'suicides_per_100k'].values.ravel()

##grouped datset withtout the suicide rate column
y_pred_grouped3 = grouped3.loc[:, grouped3.columns != 'suicides_per_100k'].values

#getting predictions of only 33 year old male group
y_pred7 = linear_oneHot2.predict(y_pred_grouped3)

In [46]:
print(f"MAE of Linear Model using One_Hot Encoding of Entire Dataset: {mae(y3, y_pred6):.4f}\n")

print(f"MAE of Linear Model using One_Hot Encoding of 33 Year Old Male Group: {mae(y_grouped3, y_pred7):.4f}\n")

MAE of Linear Model using One_Hot Encoding of Entire Dataset: 15.8347

MAE of Linear Model using One_Hot Encoding of 33 Year Old Male Group: 11.7911



### Numerical Encoding

In [47]:
#prediction input - 2-Dimensional Array
input4 = np.array([[year_mean2, 1, 33, GDP_mean2,4]])

#new prediction of suicide rate of someone who is a 33 year old male
y_pred8 = linear_numerical.predict(input4)

print(f"A 33 year old male would have a predicted suicide rate of: {y_pred8.item(0):.4f}")

A 33 year old male would have a predicted suicide rate of: 17.8755


In [48]:
grouped4 = df_numerical.copy()
#making group of 33 year old males
grouped4 = grouped4[(grouped4.sex == 1) & (grouped4.age == 30)]

#taking entire grouped dataset
y_grouped4 = grouped4.loc[:, grouped4.columns == 'suicides_per_100k'].values.ravel()
#grouped datset without suicide rate column
y_pred_grouped4 = grouped4.loc[:, grouped4.columns != 'suicides_per_100k'].values

#getting predictions of only 33 year old male group
y_pred9 = linear_numerical.predict(y_pred_grouped4)

In [49]:
print(f"MAE of Linear Model using Numerical Encoding of Entire Dataset: {mae(y2, y_pred8):.4f}\n")

print(f"MAE of Linear Model using One_Hot Encoding of 33 Year Old Male Group: {mae(y_grouped4, y_pred9):.4f}\n")

MAE of Linear Model using Numerical Encoding of Entire Dataset: 14.9028

MAE of Linear Model using One_Hot Encoding of 33 Year Old Male Group: 11.4927



The numerically encoded model predicted that a 33 year old male who is from the alpha generation would have a suicide rate of 17.8755 suicides/100k whereas the the one-hot encoded predicted the rate to be 19.5515 suicides/100k. The numerically encoded model had a slighly better MAE and was much easier to use since no extra information had to be added to the dataset.  However, in order to use the one-hot encoded method, an extra column had to be added to the dataset to account for the alpha generation.

## Problem 5

*Give one advantage when using regression (as opposed to classification with nominal features) in terms of input 
data features.*

One major advantage when using regression, as opposed to classification with nominal factors, is that it is not necessary to normalize values through binning or any other method.  For example, in our dataset, the age values are already binned for us. However, if we just had a list of ages, we could use the column as is if using linear regression.  However with classification, it would be necessary to bin the ages into categories and then to encode the ages.

## Problem 6

*Give one advantage when using regular numerical values rather than one-hot encoding for regression.*

One advantage using numerical vaues rather than one-hot encoding for regression is that when making predictions, if the prediction input includes a value that is not reflected in the dataset, the model is still able to make a prediction. You just have to adjust the input numbers to whatever you want to predict. Also, making changes to the prediction input is easier since you have much fewer columns.

However, for one-hot encoded values, you need to pay attention to having the correct input column so that you are actually making the prediction you want. I believe making such changes to the input can easily cause more errors than with the numerical values.  You also may need to add additional variables/columns and then make sure that the input is adjusted for the additional variable/column. 

For example, in Problem 4, we were predicing the suicide rates for males that are 33 and of the alpha generation.  For the numerical value prediction input, I just had to adjust the age to 33, and the generation to 7 since alpha is the generation after Z.  However, for one-hot encoding, I had to change a column to reflect age 33.  Then I had to add an additional column for Generation Z and then make sure that the input corresponded correctly to the new set of columns.

## Problem 7

*Now that you developed both a classifier and a regression model for this problem, which method do you suggest to your 
machine learning model customer and why?*

I would suggest to my machine learning model customer that the regression model be used to evaluate the suicide data set.  When predicting the suicide rate using the regression model, you can get an actual value for the suicide rate, which is a more interpetable result than having an output of a High or Low suicide rate (as we get for classification).  Also, depending on the dataset that is used, less preprocessing may need to be performed if using linear regression since nominal features such as age would not need to be binned.  

I would also suggest to incorporate other information from other datasets to improve the model. The models above had R2 scores of 30% or lower and so it should definitely improved. For example, a metric such as a Ranking of the Happiness Index could be added to the model. I found this article about other measures of human development (http://hdr.undp.org/sites/default/files/comim_template_0.pdf_. One such measure that is discussed is the Ranking of Happiness index from the World Happiness Report. The index measures the happiness of a population based on variables such social support, healthy life expectancy, absence of corruption, and social freedom as well as others. It would be interesting to determine the impact of adding this variable to the dataset and whether it would help improve the performance of the model.

***
## References

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

https://stackoverflow.com/questions/18674064/how-do-i-insert-a-column-at-a-specific-column-index-in-pandas

https://www.geeksforgeeks.org/how-to-select-rows-from-a-dataframe-based-on-column-values/

https://www.geeksforgeeks.org/ml-multiple-linear-regression-using-python/

https://www.w3schools.com/python/python_ml_multiple_regression.asp

https://towardsdatascience.com/multiple-linear-regression-model-using-python-machine-learning-d00c78f1172a

https://towardsdatascience.com/machine-learning-basics-multiple-linear-regression-9c70f796e5e3

https://numpy.org/doc/stable/reference/generated/numpy.ndarray.item.html